<a href="https://colab.research.google.com/github/Buludi/Machine_Learning_Practice/blob/master/ML101November%2012.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# https://www.kaggle.com/rohitrox/healthcare-provider-fraud-detection-analysis/data#Train_Inpatientdata-1542865627584.csv

import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler,MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from datetime import datetime

%matplotlib inline

In [0]:
Data_Label=pd.read_csv("https://raw.githubusercontent.com/princetonschoolofai/M101_11-2019/master/Train-1542865627584.csv")
Data_Beneficiarydata=pd.read_csv("https://raw.githubusercontent.com/princetonschoolofai/M101_11-2019/master/Train_Beneficiarydata-1542865627584.csv")
Data_Inpatientdata=pd.read_csv("https://raw.githubusercontent.com/princetonschoolofai/M101_11-2019/master/Train_Inpatientdata-1542865627584.csv", sep=',')

Data_Outpatientdata_0=pd.read_csv("https://raw.githubusercontent.com/princetonschoolofai/M101_11-2019/master/Train_Outpatientdata-1542865627584.csv")
Data_Outpatientdata_1=pd.read_csv("https://raw.githubusercontent.com/princetonschoolofai/M101_11-2019/master/Train_Outpatientdata-1542865627584_1.csv")
Data_Outpatientdata_2=pd.read_csv("https://raw.githubusercontent.com/princetonschoolofai/M101_11-2019/master/Train_Outpatientdata-1542865627584_2.csv")
Data_Outpatientdata_3=pd.read_csv("https://raw.githubusercontent.com/princetonschoolofai/M101_11-2019/master/Train_Outpatientdata-1542865627584_3.csv")

Data_Outpatientdata = pd.concat([Data_Outpatientdata_0,Data_Outpatientdata_1,Data_Outpatientdata_2,Data_Outpatientdata_3], axis = 0)
print(Data_Outpatientdata.shape)
print(Data_Outpatientdata_0.shape)
# Load Test Dataset


(517737, 27)
(135056, 27)


In [0]:
"""
A) Inpatient Data
This data provides insights about the claims filed for those patients who are admitted in the hospitals. It also provides additional details like their admission and discharge dates and admit d diagnosis code.
"""
Data_Inpatientdata.head()

,BeneID,ClaimID,ClaimStartDt,ClaimEndDt,Provider,InscClaimAmtReimbursed,AttendingPhysician,OperatingPhysician,OtherPhysician,AdmissionDt,ClmAdmitDiagnosisCode,DeductibleAmtPaid,DischargeDt,DiagnosisGroupCode,ClmDiagnosisCode_1,ClmDiagnosisCode_2,ClmDiagnosisCode_3,ClmDiagnosisCode_4,ClmDiagnosisCode_5,ClmDiagnosisCode_6,ClmDiagnosisCode_7,ClmDiagnosisCode_8,ClmDiagnosisCode_9,ClmDiagnosisCode_10,ClmProcedureCode_1,ClmProcedureCode_2,ClmProcedureCode_3,ClmProcedureCode_4,ClmProcedureCode_5,ClmProcedureCode_6
0,BENE11001,CLM46614,2009-04-12,2009-04-18,PRV55912,26000,PHY390922,NaN,NaN,2009-04-12,7866,1068.0,2009-04-18,201,1970,4019,5853,7843,2768,71590,2724,19889,5849,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,BENE11001,CLM66048,2009-08-31,2009-09-02,PRV55907,5000,PHY318495,PHY318495,NaN,2009-08-31,6186,1068.0,2009-09-02,750,6186,2948,56400,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7092.0,NaN,NaN,NaN,NaN,NaN
2,BENE11001,CLM68358,2009-09-17,2009-09-20,PRV56046,5000,PHY372395,NaN,PHY324689,2009-09-17,29590,1068.0,2009-09-20,883,29623,30390,71690,34590,V1581,32723,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,BENE11011,CLM38412,2009-02-14,2009-02-22,PRV52405,5000,PHY369659,PHY392961,PHY349768,2009-02-14,431,1068.0,2009-02-22,067,43491,2762,7843,32723,V1041,4254,25062,40390,4019,NaN,331.0,NaN,NaN,NaN,NaN,NaN
4,BENE11014,CLM63689,2009-08-13,2009-08-30,PRV56614,10000,PHY379376,PHY398258,NaN,2009-08-13,78321,1068.0,2009-08-30,975,042,3051,34400,5856,42732,486,5119,29620,20300,NaN,3893.0,NaN,NaN,NaN,NaN,NaN


In [0]:
"""
B) Outpatient Data

This data provides details about the claims filed for those patients who visit hospitals and not admitted in it.
"""
Data_Outpatientdata.head()

,BeneID,ClaimID,ClaimStartDt,ClaimEndDt,Provider,InscClaimAmtReimbursed,AttendingPhysician,OperatingPhysician,OtherPhysician,ClmDiagnosisCode_1,ClmDiagnosisCode_2,ClmDiagnosisCode_3,ClmDiagnosisCode_4,ClmDiagnosisCode_5,ClmDiagnosisCode_6,ClmDiagnosisCode_7,ClmDiagnosisCode_8,ClmDiagnosisCode_9,ClmDiagnosisCode_10,ClmProcedureCode_1,ClmProcedureCode_2,ClmProcedureCode_3,ClmProcedureCode_4,ClmProcedureCode_5,ClmProcedureCode_6,DeductibleAmtPaid,ClmAdmitDiagnosisCode
0,BENE120646,CLM169251,1/31/2009,1/31/2009,PRV55163,50,PHY338065,NaN,NaN,78060,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN
1,BENE120646,CLM204426,2/19/2009,2/19/2009,PRV55257,90,PHY428732,NaN,NaN,79093,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN
2,BENE120646,CLM259715,3/21/2009,3/21/2009,PRV55184,30,PHY403391,NaN,NaN,4910,72190,2724,7866,V7644,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN
3,BENE120646,CLM305868,4/15/2009,4/21/2009,PRV55215,40,PHY416553,NaN,NaN,33119,7833,29590,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN
4,BENE120646,CLM368527,5/19/2009,5/19/2009,PRV55267,400,PHY361428,NaN,NaN,7264,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN


In [0]:
"""
C) Beneficiary Details Data

This data contains beneficiary KYC details like health conditions,regioregion they belong to etc.
"""
Data_Beneficiarydata.head()

,BeneID,DOB,DOD,Gender,Race,RenalDiseaseIndicator,State,County,NoOfMonths_PartACov,NoOfMonths_PartBCov,ChronicCond_Alzheimer,ChronicCond_Heartfailure,ChronicCond_KidneyDisease,ChronicCond_Cancer,ChronicCond_ObstrPulmonary,ChronicCond_Depression,ChronicCond_Diabetes,ChronicCond_IschemicHeart,ChronicCond_Osteoporasis,ChronicCond_rheumatoidarthritis,ChronicCond_stroke,IPAnnualReimbursementAmt,IPAnnualDeductibleAmt,OPAnnualReimbursementAmt,OPAnnualDeductibleAmt
0,BENE11001,1943-01-01,NaN,1,1,0,39,230,12,12,1,2,1,2,2,1,1,1,2,1,1,36000,3204,60,70
1,BENE11002,1936-09-01,NaN,2,1,0,39,280,12,12,2,2,2,2,2,2,2,2,2,2,2,0,0,30,50
2,BENE11003,1936-08-01,NaN,1,1,0,52,590,12,12,1,2,2,2,2,2,2,1,2,2,2,0,0,90,40
3,BENE11004,1922-07-01,NaN,1,1,0,39,270,12,12,1,1,2,2,2,2,1,1,1,1,2,0,0,1810,760
4,BENE11005,1935-09-01,NaN,1,1,0,24,680,12,12,2,2,2,2,1,2,1,2,2,2,2,0,0,1790,1200


In [0]:
Data.head()

,BeneID,ClaimID,Provider,InscClaimAmtReimbursed,AttendingPhysician,ClmAdmitDiagnosisCode,DeductibleAmtPaid,DiagnosisGroupCode,ClmDiagnosisCode_1,ClmDiagnosisCode_2,Days,DOB,Gender,Race,RenalDiseaseIndicator,State,County,NoOfMonths_PartACov,NoOfMonths_PartBCov,ChronicCond_Alzheimer,ChronicCond_Heartfailure,ChronicCond_KidneyDisease,ChronicCond_Cancer,ChronicCond_ObstrPulmonary,ChronicCond_Depression,ChronicCond_Diabetes,ChronicCond_IschemicHeart,ChronicCond_Osteoporasis,ChronicCond_rheumatoidarthritis,ChronicCond_stroke,IPAnnualReimbursementAmt,IPAnnualDeductibleAmt,OPAnnualReimbursementAmt,OPAnnualDeductibleAmt,PotentialFraud
0,BENE11001,CLM46614,PRV55912,26000,PHY390922,7866,1068.0,201,1970,4019,6,1943-01-01,1,1,0,39,230,12,12,1,2,1,2,2,1,1,1,2,1,1,36000,3204,60,70,Yes
1,BENE17521,CLM34721,PRV55912,19000,PHY349293,45340,1068.0,987,4240,2639,6,1913-12-01,2,1,0,39,230,12,12,1,1,1,2,1,2,2,1,2,2,2,19000,1068,100,20,Yes
2,BENE21718,CLM72336,PRV55912,17000,PHY334706,V5789,1068.0,941,V5789,4168,6,1922-10-01,1,1,0,39,600,12,12,2,2,2,2,2,2,2,1,1,2,2,17000,1068,1050,540,Yes
3,BENE22934,CLM73394,PRV55912,13000,PHY390614,71946,1068.0,506,71535,71960,6,1930-07-01,2,1,0,39,280,12,12,2,1,2,2,1,2,1,1,1,1,2,27000,2136,450,160,Yes
4,BENE24402,CLM32911,PRV55912,3000,PHY380413,1748,1068.0,593,2330,NaN,6,1925-09-01,2,2,0,39,510,12,12,1,2,2,1,2,2,2,1,2,2,2,3000,1068,8900,740,Yes


In [0]:
Data_Inpatientdata.columns

Index(['BeneID', 'ClaimID', 'ClaimStartDt', 'ClaimEndDt', 'Provider',
       'InscClaimAmtReimbursed', 'AttendingPhysician', 'OperatingPhysician',
       'OtherPhysician', 'AdmissionDt', 'ClmAdmitDiagnosisCode',
       'DeductibleAmtPaid', 'DischargeDt', 'DiagnosisGroupCode',
       'ClmDiagnosisCode_1', 'ClmDiagnosisCode_2', 'ClmDiagnosisCode_3',
       'ClmDiagnosisCode_4', 'ClmDiagnosisCode_5', 'ClmDiagnosisCode_6',
       'ClmDiagnosisCode_7', 'ClmDiagnosisCode_8', 'ClmDiagnosisCode_9',
       'ClmDiagnosisCode_10', 'ClmProcedureCode_1', 'ClmProcedureCode_2',
       'ClmProcedureCode_3', 'ClmProcedureCode_4', 'ClmProcedureCode_5',
       'ClmProcedureCode_6'],
      dtype='object')

In [0]:
Data = Data_Inpatientdata[['BeneID', 'ClaimID', 'ClaimStartDt', 'ClaimEndDt', 'Provider',
       'InscClaimAmtReimbursed', 'AttendingPhysician', 'ClmAdmitDiagnosisCode',
       'DeductibleAmtPaid', 'DiagnosisGroupCode',
       'ClmDiagnosisCode_1', 'ClmDiagnosisCode_2']]

Data['Days'] = (datetime.strptime(Data['ClaimEndDt'][0], '%Y-%m-%d') - datetime.strptime(Data['ClaimStartDt'][0], '%Y-%m-%d')).days
Data = Data.drop('ClaimStartDt', axis = 1)
Data = Data.drop('ClaimEndDt', axis = 1)
Data.head()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,BeneID,ClaimID,Provider,InscClaimAmtReimbursed,AttendingPhysician,ClmAdmitDiagnosisCode,DeductibleAmtPaid,DiagnosisGroupCode,ClmDiagnosisCode_1,ClmDiagnosisCode_2,Days
0,BENE11001,CLM46614,PRV55912,26000,PHY390922,7866,1068.0,201,1970,4019,6
1,BENE11001,CLM66048,PRV55907,5000,PHY318495,6186,1068.0,750,6186,2948,6
2,BENE11001,CLM68358,PRV56046,5000,PHY372395,29590,1068.0,883,29623,30390,6
3,BENE11011,CLM38412,PRV52405,5000,PHY369659,431,1068.0,067,43491,2762,6
4,BENE11014,CLM63689,PRV56614,10000,PHY379376,78321,1068.0,975,042,3051,6


In [0]:
print(Data_Beneficiarydata.columns)
Data_Beneficiarydata = Data_Beneficiarydata.drop('DOD', axis = 1)
print(Data.shape)
print(Data_Beneficiarydata.shape)
Data = Data.merge(Data_Beneficiarydata,how = 'inner', on ='BeneID')
print(Data.shape)


Index(['BeneID', 'DOB', 'DOD', 'Gender', 'Race', 'RenalDiseaseIndicator',
       'State', 'County', 'NoOfMonths_PartACov', 'NoOfMonths_PartBCov',
       'ChronicCond_Alzheimer', 'ChronicCond_Heartfailure',
       'ChronicCond_KidneyDisease', 'ChronicCond_Cancer',
       'ChronicCond_ObstrPulmonary', 'ChronicCond_Depression',
       'ChronicCond_Diabetes', 'ChronicCond_IschemicHeart',
       'ChronicCond_Osteoporasis', 'ChronicCond_rheumatoidarthritis',
       'ChronicCond_stroke', 'IPAnnualReimbursementAmt',
       'IPAnnualDeductibleAmt', 'OPAnnualReimbursementAmt',
       'OPAnnualDeductibleAmt'],
      dtype='object')
(40474, 11)
(138556, 24)
(40474, 34)


In [0]:
print(Data_Label.columns)
print(Data.shape)
print(Data_Label.shape)
Data = Data.merge(Data_Label,how = 'inner', on ='Provider')
print(Data.shape)

Index(['Provider', 'PotentialFraud'], dtype='object')
(40474, 34)
(5410, 2)
(40474, 35)
